In [13]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.gaussian_process.kernels import RBF

import numpy as np
import matplotlib.pyplot as plt

from cv_svm import SVM_smooth
from kernel_svm import SVM_smooth_kernel
from sampler import sample_from_logreg

import time

import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [2]:
sigma = 1e-1
lbd = 1
experiment_sizes = np.arange(10, 1600, 130)

time_taken = {
    "IACV": np.ones(len(experiment_sizes)),
    "true": np.ones(len(experiment_sizes))
}

In [3]:
for i, e in enumerate(experiment_sizes):
    X, _, y = sample_from_logreg(n=e, p=20)

    print(f"running experiments for size {e}")
    #clf = SVM_smooth(sigma=sigma, lbd=lbd)
    clf = SVM_smooth_kernel(sigma=sigma, lbd=lbd, kernel=RBF(2.5))
    true_start = time.time()
    clf.fit(X, y, cv=True)
    true_end = time.time()
    print(f"\t true CV: {true_end - true_start}s")
    
    clf = SVM_smooth(sigma=sigma, lbd=lbd)
    approx_start = time.time()
    clf.fit(X, y, cv=False, approx_cv=True)
    approx_end = time.time()
    print(f"\t approx CV: {approx_end - approx_start}s")

    time_taken["true"][i] = true_end - true_start
    time_taken["IACV"][i] = approx_end - approx_start

running experiments for size 10
	 true CV: 0.2095470428466797s
	 approx CV: 3.2065327167510986s
running experiments for size 140
	 true CV: 8.256447792053223s
	 approx CV: 2.1469523906707764s
running experiments for size 270
	 true CV: 44.441614389419556s
	 approx CV: 2.37011456489563s
running experiments for size 400
	 true CV: 137.010169506073s
	 approx CV: 2.201307773590088s
running experiments for size 530
	 true CV: 309.31474447250366s
	 approx CV: 2.179039239883423s
running experiments for size 660
	 true CV: 594.9166464805603s
	 approx CV: 2.432981252670288s
running experiments for size 790
	 true CV: 1063.8350574970245s
	 approx CV: 2.29306960105896s
running experiments for size 920


KeyboardInterrupt: 

In [14]:
fig, ax = plt.subplots()
last_index = 7
ax.plot(experiment_sizes[:last_index], time_taken["IACV"][:last_index], label="IACV", c='black')
ax.plot(experiment_sizes[:last_index], time_taken["true"][:last_index], label="True", c='green')
ax.set_xlabel("Dataset size")
ax.set_ylabel("Time taken (s)")

ws = 1.45 # 1.75
hs = 1.35 # 1.25
fig.set_size_inches(w=5.73/ws, h=3.5/hs)
ax.legend()
plt.show()
plt.savefig('kernel_smoothsvm_cv_benchmark.pgf', bbox_inches='tight', pad_inches=0)

/tmp/ipykernel_22068/3502412378.py:12: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
